In [1]:

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd


In [2]:
# Step 1: Load Data from NPZ
npz_filename = 'training_data.npz'
# Load the .npz file
data = np.load(npz_filename)
X = data['images']  # shape: (num_images, 150, 150)
y = data['labels']  # shape: (num_images,)

# Expand dimensions to include the channel axis (for grayscale images)
X = X[..., np.newaxis]  # New shape: (num_images, 150, 150, 1)

# Normalize pixel values to [0,1]
X = X.astype('float32') / 255.0

# Split the dataset into training and validation sets (80/20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Step 2: Build the CNN Model
num_classes = len(np.unique(y))  # Automatically determine the number of classes

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse loss if labels are integers
              metrics=['accuracy'])

model.summary()


c:\Users\Muhammed Favas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,292 (18.42 MB)

 Trainable params: 4,828,292 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Step 3: Data Augmentation and Training
# Data augmentation: add random rotations, shifts, and horizontal flips
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Create generators for the training data
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

# Train the model
epochs = 30
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

# Save the final model
model.save('final_model.keras')
print("Training complete and model saved as 'final_model.keras'.")


Epoch 1/30


c:\Users\Muhammed Favas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 31s 211ms/step - accuracy: 0.4652 - loss: 1.1448 - val_accuracy: 0.6203 - val_loss: 0.9579
Epoch 2/30
  1/138 ━━━━━━━━━━━━━━━━━━━━ 25s 184ms/step - accuracy: 0.7500 - loss: 0.6970

c:\Users\Muhammed Favas\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7500 - loss: 0.6970 - val_accuracy: 0.6085 - val_loss: 0.9938
Epoch 3/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 42s 302ms/step - accuracy: 0.7262 - loss: 0.7049 - val_accuracy: 0.7604 - val_loss: 0.6292
Epoch 4/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.6562 - loss: 0.7508 - val_accuracy: 0.7703 - val_loss: 0.6188
Epoch 5/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 224ms/step - accuracy: 0.7530 - loss: 0.6351 - val_accuracy: 0.7722 - val_loss: 0.5857
Epoch 6/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7188 - loss: 0.5952 - val_accuracy: 0.7595 - val_loss: 0.5779
Epoch 7/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 29s 206ms/step - accuracy: 0.7776 - loss: 0.5943 - val_accuracy: 0.7803 - val_loss: 0.5722
Epoch 8/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7500 - loss: 0.6025 - val_accuracy: 0.7830 - val_loss: 0.5961
Epoch 9/30
138/138 ━━━━━━━━━━━━━━━━━━━━ 27s 195ms/step - accuracy: 0.8045 - loss: 0.5140 - val_accurac